## Fashion-MNIST

Capas convolucionales y de agrupación en
Fashion MNIST

In [2]:
pip install keras-tuner

In [3]:
import numpy as np; import matplotlib.pyplot as plt
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras; import keras_tuner
keras.utils.set_random_seed(23); input_dim = (28, 28, 1); num_classes = 10
(x_train_val, y_train_val), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train_val = x_train_val.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train_val = np.expand_dims(x_train_val, -1)
x_test = np.expand_dims(x_test, -1)
print(x_train_val.shape, y_train_val.shape, x_test.shape, y_test.shape)
y_train_val = keras.utils.to_categorical(y_train_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train_val[:-10000]; x_val = x_train_val[-10000:]
y_train = y_train_val[:-10000]; y_val = y_train_val[-10000:]


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)


exploramos número de filtros de la primera capa; doblamos en la segunda

In [4]:
class MyHyperModel(keras_tuner.HyperModel):
  def build(self, hp):
    M = keras.Sequential()
    M.add(keras.Input(shape=(28, 28, 1)))
    filters = hp.Int("filters", min_value=8, max_value=64, step=2, sampling="log")
    M.add(keras.layers.Conv2D(filters, kernel_size=(3, 3), activation="relu"))
    M.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    M.add(keras.layers.Conv2D(2*filters, kernel_size=(3, 3), activation="relu"))
    M.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    M.add(keras.layers.Flatten())
    M.add(keras.layers.Dense(units=800, activation='relu'))
    M.add(keras.layers.Dense(10, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.00015)
    M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return M
  def fit(self, hp, M, x, y, xy_val, **kwargs):
    factor = 0.32; patience = 5
    reduce_cb = keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=factor, patience=patience, min_delta=1e-4, min_lr=1e-5)
    early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2*patience, min_delta=1e-5)
    kwargs['callbacks'].extend([reduce_cb, early_cb])
    return M.fit(x, y, batch_size=256, epochs=100, validation_data=xy_val, **kwargs)

In [5]:
tuner = keras_tuner.BayesianOptimization(
  MyHyperModel(), objective="val_accuracy", max_trials=10, executions_per_trial=1,
  overwrite=True, directory="/tmp", project_name="Fashion-MNIST")


In [ ]:
tuner.search(x_train, y_train, (x_val, y_val))


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
32                |32                |filters

Epoch 1/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 54s 267ms/step - accuracy: 0.6185 - loss: 1.3417 - val_accuracy: 0.7847 - val_loss: 0.5733 - learning_rate: 1.5000e-04
Epoch 2/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 82s 267ms/step - accuracy: 0.8055 - loss: 0.5322 - val_accuracy: 0.8261 - val_loss: 0.4771 - learning_rate: 1.5000e-04
Epoch 3/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 81s 262ms/step - accuracy: 0.8424 - loss: 0.4470 - val_accuracy: 0.8517 - val_loss: 0.4249 - learning_rate: 1.5000e-04
Epoch 4/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 82s 263ms/step - accuracy: 0.8585 - loss: 0.4036 - val_accuracy: 0.8612 - val_loss: 0.3960 - learning_rate: 1.5000e-04
Epoch 5/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 82s 262ms/step - accuracy: 0.8686 - loss: 0.3767 - val_accuracy: 0.8666 - val_loss: 0.3783 - learning_rate: 1.5000e-04
Epoch 6/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 81s 258ms/step - accuracy: 0.874

In [ ]:
tuner.results_summary(num_trials=4)

In [ ]:
num_models = 10
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=num_models)
best_models = tuner.get_best_models(num_models=num_models)
for m in range(num_models):
  values = best_hyperparameters[m].values
  score = best_models[m].evaluate(x_test, y_test, verbose=0)
  print(f'Model {m}: Hyperparameters: {values!s} Loss: {score[0]:.4} Precisión: {score[1]:.2%}')


Ejercicio: realiza un experimento similar al de MNIST con Fashion-MNIST (mal)

In [ ]:
import numpy as np; import matplotlib.pyplot as plt
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras; import keras_tuner
keras.utils.set_random_seed(23); input_dim = (28, 28, 1); num_classes = 10
(x_train_val, y_train_val), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train_val = x_train_val.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train_val = np.expand_dims(x_train_val, -1)
x_test = np.expand_dims(x_test, -1)
print(x_train_val.shape, y_train_val.shape, x_test.shape, y_test.shape)
y_train_val = keras.utils.to_categorical(y_train_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train_val[:-10000]; x_val = x_train_val[-10000:]
y_train = y_train_val[:-10000]; y_val = y_train_val[-10000:]

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
(60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)


In [ ]:
class MyHyperModel(keras_tuner.HyperModel):
  def build(self, hp):
    M = keras.Sequential()
    M.add(keras.Input(shape=(28, 28, 1)))
    filters = hp.Int("filters", min_value=8, max_value=64, step=2, sampling="log")
    M.add(keras.layers.Conv2D(filters, kernel_size=(3, 3), activation="relu"))
    M.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    M.add(keras.layers.Conv2D(2*filters, kernel_size=(3, 3), activation="relu"))
    M.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    M.add(keras.layers.Flatten())
    M.add(keras.layers.Dense(units=800, activation='relu'))
    M.add(keras.layers.Dense(10, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.00168)
    M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return M
  def fit(self, hp, M, x, y, xy_val, **kwargs):
    factor = 0.32; patience = 5
    reduce_cb = keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=factor, patience=patience, min_delta=1e-4, min_lr=1e-5)
    early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2*patience, min_delta=1e-5)
    kwargs['callbacks'].extend([reduce_cb, early_cb])
    return M.fit(x, y, batch_size=256, epochs=10, validation_data=xy_val, **kwargs)

In [ ]:
tuner = keras_tuner.BayesianOptimization(
 MyHyperModel(), objective="val_accuracy", max_trials=10, executions_per_trial=1,
 overwrite=True, directory="/tmp", project_name="MNIST")

In [ ]:
tuner.search(x_train, y_train, (x_val, y_val))

Trial 10 Complete [00h 00m 29s]
val_accuracy: 0.9083999991416931

Best val_accuracy So Far: 0.9154000282287598
Total elapsed time: 00h 04m 25s


In [ ]:
tuner.results_summary(num_trials=4)

Results summary
Results in /tmp/MNIST
Showing 4 best trials
Objective(name="val_accuracy", direction="max")

Trial 08 summary
Hyperparameters:
filters: 64
Score: 0.9154000282287598

Trial 07 summary
Hyperparameters:
filters: 64
Score: 0.911899983882904

Trial 05 summary
Hyperparameters:
filters: 64
Score: 0.9103999733924866

Trial 04 summary
Hyperparameters:
filters: 64
Score: 0.910099983215332


In [ ]:
num_models = 10
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=num_models)
best_models = tuner.get_best_models(num_models=num_models)
for m in range(num_models):
 values = best_hyperparameters[m].values
 score = best_models[m].evaluate(x_test, y_test, verbose=0)
 print(f'Model {m}: Hyperparameters: {values!s} Loss: {score[0]:.4} Precisión: {score[1]:.2%}')

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model 0: Hyperparameters: {'filters': 64} Loss: 0.2951 Precisión: 90.78%
Model 1: Hyperparameters: {'filters': 64} Loss: 0.2928 Precisión: 90.68%
Model 2: Hyperparameters: {'filters': 64} Loss: 0.344 Precisión: 90.42%
Model 3: Hyperparameters: {'filters': 64} Loss: 0.2875 Precisión: 90.94%
Model 4: Hyperparameters: {'filters': 64} Loss: 0.2976 Precisión: 90.19%
Model 5: Hyperparameters: {'filters': 32} Loss: 0.2957 Precisión: 90.29%
Model 6: Hyperparameters: {'filters': 64} Loss: 0.2961 Precisión: 90.02%
Model 7: Hyperparameters: {'filters': 64} Loss: 0.2715 Precisión: 90.10%
Model 8: Hyperparameters: {'filters': 16} Loss: 0.2829 Precisión: 89.87%
Model 9: Hyperparameters: {'filters': 8} Loss: 0.2998 Precisión: 89.13%
